In [1]:
from tinyec import registry
from Crypto.Cipher import AES
import hashlib, secrets, binascii
import tinyec.ec as ec

def encrypt_AES_GCM(msg, secretKey):
    aesCipher = AES.new(secretKey, AES.MODE_GCM)
    ciphertext, authTag = aesCipher.encrypt_and_digest(msg)
    return (ciphertext, aesCipher.nonce, authTag)

def decrypt_AES_GCM(ciphertext, nonce, authTag, secretKey):
    aesCipher = AES.new(secretKey, AES.MODE_GCM, nonce)
    plaintext = aesCipher.decrypt_and_verify(ciphertext, authTag)
    return plaintext

def ecc_point_to_256_bit_key(point):
    sha = hashlib.sha256(int.to_bytes(point.x, 32, 'big'))
    sha.update(int.to_bytes(point.y, 32, 'big'))
    return sha.digest()

def encrypt_ECC(msg, pubKey):
    curve = registry.get_curve('brainpoolP256r1')
    ciphertextPrivKey = secrets.randbelow(curve.field.n)
    sharedECCKey = ciphertextPrivKey * pubKey
    secretKey = ecc_point_to_256_bit_key(sharedECCKey)
    ciphertext, nonce, authTag = encrypt_AES_GCM(msg, secretKey)
    ciphertextPubKey = ciphertextPrivKey * curve.g
    return (ciphertext, nonce, authTag, ciphertextPubKey)

def decrypt_ECC(encryptedMsg, privKey):
    (ciphertext, nonce, authTag, ciphertextPubKey) = encryptedMsg
    sharedECCKey = privKey * ciphertextPubKey
    secretKey = ecc_point_to_256_bit_key(sharedECCKey)
    plaintext = decrypt_AES_GCM(ciphertext, nonce, authTag, secretKey)
    return plaintext

In [2]:
###### KEY GEN SCRIPT ##########
from tinyec import registry
from Crypto.Cipher import AES
import hashlib, secrets, binascii
import tinyec.ec as ec

# convert pubKey thing to hex
def compress_point(point):
    return hex(point.x) + hex(point.y)

# generate keys
def keyGen():
    
    # obviously you can change the curve values
    curve = registry.get_curve('brainpoolP256r1')
    
    privKey = secrets.randbelow(curve.field.n)
    pubKey = privKey * curve.g
    
    return privKey, pubKey
    
    
privKey, pubKey = keyGen()

privKeyHex = hex(privKey)
pubKeyHex = compress_point(pubKey)


with open("privKey.txt", "w") as privk:
    privk.write(str(privKeyHex))

with open("pubKey.txt", "w") as pubk:
    pubk.write(str(pubKeyHex))
    
print("private key: " + str(privKeyHex))
print("public key: " + str(pubKeyHex))

private key: 0xf0e5d568ff4a1d7368f8be29d9ae97ee7936a8a36fea1009f2e8577048086a1
public key: 0x437b792a0ed5d4df63fdf7f5caa521fc6f771d5e8709cb58d9188772811f95c30x99ec81558673cbef5060ccdbe3014c334e6698f095f51fc7cb63405a4342ea10


In [3]:
# compress x and y values to hex
def compress_point(point):
    pubKeyHex = hex(point.x) + hex(point.y)
    return pubKeyHex

# convert pubKeyHex to ec.Point
def decompress_point(curve, pubKeyHex):
    n = 0
    x = ""
    y = ""

    l = len(str(pubKeyHex))
    length = l/2
    
    for i in pubKeyHex:
        if n < length:
            x+=i
        else:
            y+=i     
        n+=1
        
    x = int(x,16)
    y = int(y,16)
    
    p = ec.Point(curve,x,y)
    
    return p

In [4]:
# initial user input 
def messageInput():
    message = str(input("Message: "))
    return message
    

def encryptUserInput(message, pubKey):
    msg = message.encode("ascii")
    
    # in the future users can choose which curve
    curve = registry.get_curve('brainpoolP256r1')
    
    encryptedMsg = encrypt_ECC(msg, pubKey)

    # put encryptedMsg in dictionary
    encryptedMsgDic = {
    'ciphertext': binascii.hexlify(encryptedMsg[0]),
    'nonce': binascii.hexlify(encryptedMsg[1]),
    'authTag': binascii.hexlify(encryptedMsg[2]),
    'x': hex(encryptedMsg[3].x),
    'y': hex(encryptedMsg[3].y)
    }
    # convert all vals in encryptedMsgDic to str
    a = str(encryptedMsgDic["ciphertext"])
    b = str(encryptedMsgDic["nonce"])
    c = str(encryptedMsgDic["authTag"])
    d = str(encryptedMsgDic["x"])
    e = str(encryptedMsgDic["y"])

    encryptedMsgDicStr = {
        'a': a,
        'b': b,
        'c': c,
        'd': d,
        'e': e
    }
    
    # convert encryptedMsgDicStr to string with : as separation of vals
    cipherstr = ''
    for i in encryptedMsgDicStr:
        cipherstr += encryptedMsgDicStr[i] + ' : '

    #encode cipherstr and send it  
    m = cipherstr.encode("ascii")
    
    return m



In [5]:
def connector():

        # the pubKey is essentially the address of the other user
    with open("pubKey.txt", "r") as pubk:
        pubKeyHex = pubk.read()
        
    # get PUBLIC KEY POINT
    curve = registry.get_curve('brainpoolP256r1')
    
    pubKey = decompress_point(curve,pubKeyHex)

    message = messageInput()
    encryptedMsgS = encryptUserInput(message, pubKey)

    return encryptedMsgS



message = '{}~ {}'.format(input("To: "), connector())


message = message.encode('ascii')

print(message)

################################ SEND THROUGH SERVER #####################

To: asdfasdf
Message: asdfasdfasdfasdfasdfasdf
b'asdfasdf~ b"b\'77e1348d17a755cb39ae0587904c1c041086d793de455c94\' : b\'27b0ca61438c69aa82cd1efed4a51c48\' : b\'abc371654276d11c5e817089aa8aea5a\' : 0x620d7c1d2533f1785c177a5bf4e816916b44403851a2ff97c8d2e71f40c580ed : 0x5cb980191cb5cd7c7fcf8109df835747b712d9fa9e609f6ccc5069365af985f2 : "'


In [6]:
############### SERVER #################

In [16]:
from subprocess import run
from ecies import encrypt, decrypt


def cleanData(message):
    data = message
    m = data.decode("utf-8")
    l = m.split('~')

    user = l[0]
    text = l[1]
    
    text = eval(text)

    return user, text



# server side upload function
def uploadIPFS(message):
    with open("txt.txt", "wb") as txt:
        txt.write(message)
    cmd = [ 'ipfs', 'add', 'txt.txt' ]
    out = run(cmd, capture_output=True).stdout
    output = out.decode("utf-8")
    outputs = output.split(" ")
    uri = outputs[1]
    return uri


In [17]:
user, text = cleanData(message)
uri = uploadIPFS(text)

print(type(uri))

<class 'str'>


In [9]:
########## CLIENT ##############

In [10]:
### download from IPFS
def ipfs(uri):
    cmd = [ 'ipfs', 'get', uri ]
    out = run(cmd, capture_output=True).stdout
    output = out.decode("utf-8")
    outputs = output.split(" ")
    file = outputs[3]
    file = file.rstrip()
    return file
    
# opening file
def dec(file):
    with open(file, 'rb') as file:
        cipher = file.read()
        return cipher

In [11]:
# rebuild data structure
def decryptMessage(m):
    d = m.decode("ascii")
    
    array = d.split(': ')
    
    print(array)
    
    #a = array[0].replace("\\", "")
    
    a = array[0].strip()
    a = eval(a)
    ciphertext = binascii.unhexlify(a)

    b = array[1].strip()
    b = eval(b)
    nonce = binascii.unhexlify(b)

    c = array[2].strip()
    c = eval(c)
    authTag = binascii.unhexlify(c)

    x = array[3].strip()
    y = array[4].strip()
    
    # convert x,y to int 
    x = int(x,16)
    y = int(y,16)
    
    curve = registry.get_curve('brainpoolP256r1')
    
    p = ec.Point(curve,x,y)
    
    array = [ciphertext, nonce, authTag, p]
    
    encryptedMsgS = tuple(array)
    
    return encryptedMsgS

In [12]:



def connector2(uri):
    file = ipfs(uri)
    cipher = dec(file)
    encryptedMsgS = decryptMessage(cipher)
    
    with open("privKey.txt", "r") as privK:
        privKeyHex = privK.read()

    privKey = int(privKeyHex,16)
    
    message = decrypt_ECC(encryptedMsgS, privKey)
    
    print(message)
    
    return message

connector2()

["b'77e1348d17a755cb39ae0587904c1c041086d793de455c94' ", "b'27b0ca61438c69aa82cd1efed4a51c48' ", "b'abc371654276d11c5e817089aa8aea5a' ", '0x620d7c1d2533f1785c177a5bf4e816916b44403851a2ff97c8d2e71f40c580ed ', '0x5cb980191cb5cd7c7fcf8109df835747b712d9fa9e609f6ccc5069365af985f2 ', '']
b'asdfasdfasdfasdfasdfasdf'


b'asdfasdfasdfasdfasdfasdf'

In [1]:
###############3

In [30]:
import os
import sys
import platform
import pip
from subprocess import getoutput
#from modules import banner
#from modules import localhost



if platform.uname()[0] == "Windows":
    print("\n This Tool Only Works On Linux Distributions\n")
    exit()
else:
    pass

root = os.environ.get('USER')
if root != "root":
    exit("You need to have root privileges to run this script.\nPlease try again, this time using 'sudo'. Exiting.")


check_php = getoutput("php -v")
if "not found" in check_php:
    exit("please install php \n command > sudo apt install php")

try:
    from colorama import Fore 
    import requests
    from pyngrok import ngrok
    
except ImportError:
    print("please install library \n command > python3 -m pip install -r requirments.txt")


while True:
    banner.banner()
    banner.infolist0()
    

    try:

        input1 = input(Fore.RED+" ┌─["+Fore.LIGHTGREEN_EX+"STORM-BREAKER"+Fore.BLUE+"~"+Fore.WHITE+"@HOME"+Fore.RED+"""]
 └──╼ """+Fore.WHITE+"$ ")
        
        if input1 == "1":
            localhost.webcham()
        
        elif input1 == "2":
            banner.banner()   
            localhost.micro()
        
        elif input1 == "3":
            banner.banner()
            localhost.screen()

        

        elif input1 == "4":
            banner.banner()
            localhost.location()
        
        elif input1 == "5":
            banner.banner()
            banner.Settings()

        elif input1 == "6":
            print("\n God Lock :) ")
            sys.exit()

        
            



    except KeyboardInterrupt:
        print("")
        sys.exit()

NameError: name 'banner' is not defined

In [41]:
from subprocess import run

def ipfs(uri):

    path = "URIs/"

    cmd = [ 'ipfs', 'get', uri, f"--output={path}{uri}"]
    out = run(cmd, capture_output=True).stdout
    output = out.decode("utf-8")
    outputs = output.split(" ")
    file = outputs[3]
    file = file.rstrip()
    return file


ipfs("QmRo6UuXT5SoRbXHv12v5E1grrJVENYYnBUfjmkaJEBdgZ")

'URIs/QmRo6UuXT5SoRbXHv12v5E1grrJVENYYnBUfjmkaJEBdgZ'

In [40]:
def ipfs(uri):
    cmd = [ 'ipfs', 'get', uri ]
    out = run(cmd, capture_output=True).stdout
    output = out.decode("utf-8")
    outputs = output.split(" ")
    file = outputs[3]
    file = file.rstrip()
    return file


ipfs("QmRo6UuXT5SoRbXHv12v5E1grrJVENYYnBUfjmkaJEBdgZ")

'QmRo6UuXT5SoRbXHv12v5E1grrJVENYYnBUfjmkaJEBdgZ'